<a href="https://colab.research.google.com/github/monsecc01/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"

spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

Clean the data

In [6]:
clean_df = df.dropna()

In [7]:
clean_df =df.drop_duplicates()

Recreate the vine_table and perform analysis

In [8]:
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1000E0AHJ5E1V|          4|            0|          0|   N|                N|
|R100A5CHCA9GMG|          1|            2|          2|   N|                Y|
|R100A968SZR4GH|          5|            0|          0|   N|                Y|
|R100HVIG9VEABL|          5|            1|          1|   N|                N|
|R100ZMBT0DGFFH|          5|            8|         10|   N|                Y|
|R101BSLZ8K12T6|          5|            0|          1|   N|                Y|
|R101I5DOJAUSZ7|          4|            0|          0|   N|                Y|
|R101VVQ9LXBADV|          5|            1|          1|   N|                Y|
|R101VY19JHVABI|          4|            0|          0|   N|                N|
|R1026R3QQEM02Q|          5|            0|          0|   N|     

In [9]:
vine_df.count()

4366916

In [11]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20

from pyspark.sql.functions import col
twenty_vine_df = vine_df.filter(col("total_votes") >= 20)
twenty_vine_df.show(10)

twenty_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RFO47EUA8VO07|          1|          153|        165|   N|                Y|
|R1FY7QX8ZVKF3B|          2|           23|         27|   N|                Y|
|R1JQ6FVMNPTTGJ|          5|           58|         58|   N|                Y|
|R3TGDQWQ4IWBF4|          5|           18|         20|   N|                Y|
| RFIFX28ALACM4|          1|           49|         51|   N|                Y|
|R31YTDENNMDYLJ|          1|           40|         40|   N|                Y|
|R17UV33XE6IWTX|          1|           17|         22|   N|                Y|
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R2SWEEB6CUQ358|          3|           36|         37|   N|                Y|
|R320QZ1YWKW6KC|          4|           22|         24|   N|     

27884

In [12]:
# Filter new dataframe to retrieve helpful_votes/total_votes => 50

fifty_vine_df = twenty_vine_df.filter((twenty_vine_df.helpful_votes / twenty_vine_df.total_votes)>=0.5)
fifty_vine_df.show(10)

fifty_vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RFO47EUA8VO07|          1|          153|        165|   N|                Y|
|R1FY7QX8ZVKF3B|          2|           23|         27|   N|                Y|
|R1JQ6FVMNPTTGJ|          5|           58|         58|   N|                Y|
|R3TGDQWQ4IWBF4|          5|           18|         20|   N|                Y|
| RFIFX28ALACM4|          1|           49|         51|   N|                Y|
|R31YTDENNMDYLJ|          1|           40|         40|   N|                Y|
|R17UV33XE6IWTX|          1|           17|         22|   N|                Y|
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R2SWEEB6CUQ358|          3|           36|         37|   N|                Y|
|R320QZ1YWKW6KC|          4|           22|         24|   N|     

27009

In [14]:
paid_df = fifty_vine_df.filter(col("vine") == 'Y')
paid_df.show(10)

paid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R11XKHFS4KQS3Z|          4|          205|        211|   Y|                N|
|R3A7BQX1JDKOM5|          4|           20|         20|   Y|                N|
|R1307JMPUEQXOW|          5|           31|         35|   Y|                N|
|R3SEZS7BZEC69Y|          5|           16|         20|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
| RBMDYE7LUH9FI|          5|           26|         32|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R3KOK2SH39BZU1|          3|           94|         96|   Y|     

22

In [16]:
five_star_paid_df = paid_df.filter(col("star_rating") == 5)

five_star_paid_df.count()

13

In [21]:
percentage_paid = (five_star_paid_df.count()/paid_df.count())*100
percentage_paid

59.09090909090909

In [15]:
unpaid_df = fifty_vine_df.filter(col("vine") == 'N')
unpaid_df.show(10)

unpaid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RFO47EUA8VO07|          1|          153|        165|   N|                Y|
|R1FY7QX8ZVKF3B|          2|           23|         27|   N|                Y|
|R1JQ6FVMNPTTGJ|          5|           58|         58|   N|                Y|
|R3TGDQWQ4IWBF4|          5|           18|         20|   N|                Y|
| RFIFX28ALACM4|          1|           49|         51|   N|                Y|
|R31YTDENNMDYLJ|          1|           40|         40|   N|                Y|
|R17UV33XE6IWTX|          1|           17|         22|   N|                Y|
| R701I3ZQH3MYJ|          5|           49|         51|   N|                Y|
|R2SWEEB6CUQ358|          3|           36|         37|   N|                Y|
|R320QZ1YWKW6KC|          4|           22|         24|   N|     

26987

In [17]:
five_star_unpaid_df = unpaid_df.filter(col("star_rating") == 5)

five_star_unpaid_df.count()

14475

In [23]:
percentage_unpaid= (five_star_unpaid_df.count()/unpaid_df.count())*100
percentage_unpaid

53.63693630266425